In [7]:
from ray.tune.registry import register_env
from pycigar.utils.registry import make_create_env
import yaml
import argparse
import ray
from ray import tune

parser = argparse.ArgumentParser()
parser.add_argument("--run", type=str, default="PPO")  # try PG, PPO, IMPALA
parser.add_argument("--stop", type=int, default=0)

stream = open("../rl_config_scenarios.yaml", "r")
sim_params = yaml.safe_load(stream)

pycigar_params = {"exp_tag": "cooperative_multiagent_ppo",
                  "env_name": "SingleDiscreteCoopEnv",
                  "sim_params": sim_params,
                  "simulator": "opendss",
                  "tracking_ids": ['pv_8', 'pv_9', 'pv_12']}

In [ ]:
create_env, env_name = make_create_env(params=pycigar_params, version=0)
register_env(env_name, create_env)

test_env = create_env()
obs_space = test_env.observation_space
act_space = test_env.action_space

In [6]:
if __name__ == "__main__":
    ray.init()
    args = parser.parse_args()
    tune.run(
        args.run,
        stop={"episode_reward_mean": args.stop},
        config={
            "env": env_name,
            "gamma": 0.5,
            'model': {'conv_filters': None, 'conv_activation': 'relu',
                      'fcnet_activation': 'tanh', 'fcnet_hiddens': [256, 256],
                      'free_log_std': False, 'no_final_linear': False, 'vf_share_layers': True,
                      'use_lstm': False, 'max_seq_len': 20, 'lstm_cell_size': 256,
                      'lstm_use_prev_action_reward': False, 'state_shape': None,
                      'framestack': False, 'dim': 84, 'grayscale': False,
                      'zero_mean': True, 'custom_preprocessor': None,
                      'custom_model': None, 'custom_action_dist': None, 'custom_options': {}},
            "num_gpus": 0,
            'multiagent': {
                "policies": {
                    "pol": (None, obs_space, act_space, {}),
                },
                "policy_mapping_fn": lambda x: "pol",
            }
        })


({'pv_8': array([[ 1.02326127e+00,  1.02326260e+00,  1.02326330e+00,
           1.02326335e+00],
         [ 5.82013299e+01,  5.81659590e+01,  5.81280250e+01,
           5.80874645e+01],
         [ 2.31001854e-02,  2.09002282e-02,  1.89097663e-02,
           1.71088536e-02],
         [-5.82408090e+01, -5.82345925e+01, -5.82250662e+01,
          -5.82123079e+01],
         [ 1.46179796e+01,  1.46428003e+01,  1.46682274e+01,
           1.46943302e+01]]),
  'pv_9': array([[ 1.02325008e+00,  1.02325146e+00,  1.02325221e+00,
           1.02325231e+00],
         [ 6.58782460e+01,  6.59105818e+01,  6.59437719e+01,
           6.59778329e+01],
         [ 2.30862125e-02,  2.08875883e-02,  1.88983321e-02,
           1.70985097e-02],
         [-6.55308754e+01, -6.55704092e+01, -6.56092066e+01,
          -6.56474416e+01],
         [ 2.37988380e+01,  2.38025604e+01,  2.38043802e+01,
           2.38043341e+01]])},
 {'pv_8': -0.00029253595240493445, 'pv_9': -0.00029253595240493445},
 {'__all__': False},